After failing to get the IBM Lab for this assignment working, I decided to complete it all inside a Jupyter Labs Notebook
-the Dashboard is below the python script once the cell is run -you can also use the Dash app at http://127.0.0.1:8050/ if run on a local server

In [ ]:
pip install pandas
pip install Dash
pip install plotly

In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import Dash, html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                    dcc.Dropdown(
                                                id='site-dropdown',
                                                options=[
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                ],
                                                placeholder="All Sites",
                                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.H2("Total Success Launches by Site", style={'textAlign': 'center'}),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(
                                                id='payload-slider',
                                                min=min_payload,
                                                max=max_payload,
                                                step=1,
                                                marks={
                                                int(min_payload): str(int(min_payload)),
                                                int(max_payload): str(int(max_payload))
                                                },
                                                value=[min_payload, max_payload],
                                                ),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    prevent_initial_call=False
)
def get_pie_chart(selected_option):
    if selected_option is None:
        # Group by 'Launch Site', sum 'class'
        summary = spacex_df.groupby('Launch Site')['class'].sum().reset_index()

        fig = px.pie(
            summary,
            names='Launch Site',
            values='class',
            title="Total Success Launches By Site"
        )
        return fig

    filtered_df = spacex_df[spacex_df['Launch Site'] == selected_option]
    counts = filtered_df['class'].value_counts().sort_index()
    values = counts.values
    labels = ['Failure', 'Success'] if len(counts) == 2 else ['Success']

    fig = px.pie(
        names=labels,
        values=values,
        title=f"Success and Failure for {selected_option}"
    )
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='payload-slider', component_property='value'),
     Input(component_id='site-dropdown', component_property='value')],
    prevent_initial_call=False  # forces the function to run on startup to show the All sites
)
def update_scatter(slider_range, selected_option):
    low, high = slider_range

    # Filter by Payload Mass (kg) column range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]

    # Further filter by dropdown selection if selected
    if selected_option is not None:
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_option]

    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title=f"Correlation between Payload and Success"
              + (f" for {selected_option}" if selected_option else " (all Sites)")
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run()

